# Inference Review : different models

## About

- Look at timing information of different bert models

## imports

In [1]:
import pathlib
import sklearn
import datasets
import pandas as pd
import torch

import numpy as np
import transformers
import os
import json
from ts.utils.util  import map_class_to_label
from tqdm import tqdm, trange
import time
import torchviz
import torch.nn as nn


In [2]:
query = "men shoes"

## Device Specs

In [3]:
print ( torch.__config__.parallel_info())

ATen/Parallel:
	at::get_num_threads() : 8
	at::get_num_interop_threads() : 8
OpenMP 201511 (a.k.a. OpenMP 4.5)
	omp_get_max_threads() : 8
Intel(R) oneAPI Math Kernel Library Version 2022.1-Product Build 20220311 for Intel(R) 64 architecture applications
	mkl_get_max_threads() : 8
Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
std::thread::hardware_concurrency() : 16
Environment variables:
	OMP_NUM_THREADS : [not set]
	MKL_NUM_THREADS : [not set]
ATen parallel backend: OpenMP



## Model: bert-large-uncased

In [4]:
base_model = "bert-large-uncased"

In [5]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(base_model)

tokenizer = transformers.AutoTokenizer.from_pretrained(
                base_model
            )

res = tokenizer.encode_plus(query, return_tensors="pt",  padding="max_length", truncation=True)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [6]:
res

{'input_ids': tensor([[ 101, 2273, 6007,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [7]:
%%timeit -r 3 -n 5

model_res = model(**res)
model_res

734 ms ± 25.5 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [8]:
%%timeit -r 3 -n 5
if torch.cuda.is_available():
    model_cuda = model.cuda()
    model_cuda(res['input_ids'].cuda(),res['attention_mask'].cuda())

The slowest run took 5.28 times longer than the fastest. This could mean that an intermediate result is being cached.
278 ms ± 228 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


## Model: bert-base-uncased

In [9]:
base_model = "bert-base-uncased"

In [10]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(base_model)

tokenizer = transformers.AutoTokenizer.from_pretrained(
                base_model
            )

res = tokenizer.encode_plus(query, return_tensors="pt",  padding="max_length", truncation=True)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [11]:
%%timeit -r 3 -n 5

model_res = model(**res)
model_res

217 ms ± 6.94 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [12]:
%%timeit -r 3 -n 5
if torch.cuda.is_available():
    model_cuda = model.cuda()
    model_cuda(res['input_ids'].cuda(),res['attention_mask'].cuda())

43.8 ms ± 13.9 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


## Model: distilbert-base-uncased

In [13]:
base_model = "distilbert-base-uncased"

In [14]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(base_model)

tokenizer = transformers.AutoTokenizer.from_pretrained(
                base_model
            )

res = tokenizer.encode_plus(query, return_tensors="pt",  padding="max_length", truncation=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [15]:
%%timeit -r 3 -n 5

model_res = model(**res)
model_res

112 ms ± 4.13 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [16]:
%%timeit -r 3 -n 5
if torch.cuda.is_available():
    model_cuda = model.cuda()
    model_cuda(res['input_ids'].cuda(),res['attention_mask'].cuda())

26.6 ms ± 6.81 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
